In [1]:
%pip install torch torchvision
%pip install ftfy regex tqdm
%pip install googletrans==4.0.0-rc1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install GitPython

Note: you may need to restart the kernel to use updated packages.


In [3]:
from git import Repo
import os

CLIP_repo_url = "https://github.com/openai/CLIP.git"
support_models_path = "/home/students/ttnhan-cse23/hcmai/support_models"
CLIP_target_path = os.path.join(support_models_path, "CLIP")

Repo.clone_from(CLIP_repo_url, CLIP_target_path)
print(f"Cloned CLIP repo to {CLIP_target_path}")

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/openai/CLIP.git /home/students/ttnhan-cse23/hcmai/support_models/CLIP
  stderr: 'fatal: destination path '/home/students/ttnhan-cse23/hcmai/support_models/CLIP' already exists and is not an empty directory.
'

In [4]:
import sys
sys.path.append("/home/students/ttnhan-cse23/hcmai")

### Test Translator

In [5]:
from googletrans import Translator

translator = Translator()

text = "Xin chào, bạn khỏe không?"

# Detect the language
detected = translator.detect(text)

if detected.lang == "vi":
    translated = translator.translate(text, src="vi", dest="en")
    print("Translated text:", translated.text)
else:
    print("The text is not in Vietnamese. Detected language: ", detected.lang)

Translated text: Hello, how are you?


### CLIP Encoder

In [22]:
import torch
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize

from support_models.CLIP.clip import clip

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm
import os
import sys

In [7]:
# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14@336px", device=device)

In [8]:
print(device)

cuda


In [9]:
# ENCODE TEXT AND IMAGE
def encode_text(text):
    text = clip.tokenize([text]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text)
    return text_features

def encode_images(images):
    with torch.no_grad():
        image_features = torch.cat([model.encode_image(image) for image in images])
    return image_features

In [17]:
extracted_keyframes_dir = "/home/students/ttnhan-cse23/hcmai/dataset/extracted_frames"

all_keyframe_paths = dict()
for data_part in sorted(os.listdir(extracted_keyframes_dir)):
    data_part = data_part.split('_')[-1] # L01, L02 for ex
    all_keyframe_paths[data_part] = dict()
print((all_keyframe_paths))

for data_part in sorted(all_keyframe_paths.keys()):
    data_part_path = f'{extracted_keyframes_dir}/{data_part}'
    video_dirs = sorted(os.listdir(data_part_path))
    video_ids =  [video_dir.split('_')[-1] for video_dir in video_dirs]
    for video_id, video_dir in zip(video_ids, video_dirs):
        keyframe_paths = sorted(glob.glob(f'{data_part_path}/{video_dir}/*.jpg'))
        all_keyframe_paths[data_part][video_id] = keyframe_paths
print(all_keyframe_paths)

{'L01': {}}
{'L01': {'V001': ['/home/students/ttnhan-cse23/hcmai/dataset/extracted_frames/L01/V001/000000.jpg', '/home/students/ttnhan-cse23/hcmai/dataset/extracted_frames/L01/V001/000002.jpg', '/home/students/ttnhan-cse23/hcmai/dataset/extracted_frames/L01/V001/000005.jpg', '/home/students/ttnhan-cse23/hcmai/dataset/extracted_frames/L01/V001/000006.jpg', '/home/students/ttnhan-cse23/hcmai/dataset/extracted_frames/L01/V001/000027.jpg', '/home/students/ttnhan-cse23/hcmai/dataset/extracted_frames/L01/V001/000049.jpg', '/home/students/ttnhan-cse23/hcmai/dataset/extracted_frames/L01/V001/000050.jpg', '/home/students/ttnhan-cse23/hcmai/dataset/extracted_frames/L01/V001/000229.jpg', '/home/students/ttnhan-cse23/hcmai/dataset/extracted_frames/L01/V001/000409.jpg', '/home/students/ttnhan-cse23/hcmai/dataset/extracted_frames/L01/V001/000410.jpg', '/home/students/ttnhan-cse23/hcmai/dataset/extracted_frames/L01/V001/000438.jpg', '/home/students/ttnhan-cse23/hcmai/dataset/extracted_frames/L01/V001

#### Encode and Save in `.npy` format

In [ ]:
batch_size = 4
save_clip_dir = "/home/students/ttnhan-cse23/hcmai/dataset/CLIP_features"

if not os.path.exists(save_clip_dir):
  os.mkdir(save_clip_dir)

for key, video_keyframe_paths in all_keyframe_paths.items():
  video_ids = sorted(video_keyframe_paths.keys())

  if not os.path.exists(f'{save_clip_dir}/{key}'):
      os.mkdir(os.path.join(save_clip_dir,key))

  for video_id in tqdm(video_ids):
      video_feats = []
      video_keyframe_path = video_keyframe_paths[video_id]
      for i in range(0, len(video_keyframe_path), batch_size):
          # Support batchsize inferencing
          images = []
          image_paths = video_keyframe_path[i:i+batch_size]
          for image_path in image_paths:
            try:
              image = preprocess(Image.open(image_path)).unsqueeze(0)
              images.append(image)
            except FileNotFoundError as e:
              print(e)
              pass

          if images :
            images = torch.cat(images).to(device)

            with torch.no_grad():
                image_feats = model.encode_image(images)
            image_feats /= image_feats.norm(dim=-1, keepdim=True)
            for b in range(image_feats.shape[0]):
                video_feats.append(image_feats[b].detach().cpu().numpy().astype(np.float32).flatten())
      np.save(f'{save_clip_dir}/{key}/{video_id}.npy', video_feats)

100%|██████████| 5/5 [01:06<00:00, 13.32s/it]


### Faiss

In [15]:
%pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 5.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [13]:
import faiss

def save_faiss_index(images, index_path):
    image_features = encode_images(images).cpu().numpy()
    
    # Normalize features
    image_features = image_features / np.linalg.norm(image_features, axis=1, keepdims=True)
    
    # Create FAISS Index
    index = faiss.IndexFlatIP(image_features.shape[1])
    index.add(image_features)
    faiss.write_index(index, index_path)
    
def load_faiss_index(index_path):
    index = faiss.read_index(index_path)
    return index

In [14]:
# Display the most similar images
def display_similar_images(image_paths, index, text, num_images=5):
    print(f"Text Query: {text}")
    translator = Translator()
    detected = translator.detect(text)
    if detected.lang == "vi":
        translated = translator.translate(text, src="vi", dest="en")
        text = translated.text
        print(f"Translated Query: {text}")
    text_features = encode_text(text)
    text_features = text_features.cpu().numpy()
    D, I = index.search(text_features, k=len(image_paths))
    
    sorted_image_paths = [image_paths[i] for i in I[0]]
    sorted_similarities = D[0]
    
    print("Top matches: ")
    
    plt.figure(figsize=(20, 5))
    for i in range(min(num_images, len(sorted_image_paths))):
        img = Image.open(sorted_image_paths[i])
        plt.subplot(1, num_images, i+1)
        plt.imshow(img)
        plt.title(f"Score: {sorted_similarities[i]:.2f}")
        plt.axis("off")
    plt.show()

### Test

In [24]:
CLIP_features_dir = "/home/students/ttnhan-cse23/hcmai/dataset/CLIP_features"
CLIP_features_folders = os.listdir(CLIP_features_dir)
CLIP_features_folders.sort()
# CLIP_features_folders = CLIP_features_folders[:-1]
CLIP_features_folders

['L01']

In [ ]:
vector_list_CLIP = []
num_clip = 0

for folder in CLIP_features_folders:
    folder_path = os.path.join(CLIP_features_dir, folder)
    videos_features = os.listdir(folder_path)
    videos_features.sort()
    print("Processing folder: {folder}")
    for video_feature in videos_features:
        npy_file = os.path.join(folder_path, video_feature)
        if npy_file.endswith(".npy"):
            vector = np.load(npy_file)
            num_clip += vector.shape[0]
            vector_list_CLIP.append(vector)
            print(f"Loaded vector from {npy_file}")

Processing folder: {folder}
Loaded vector from /home/students/ttnhan-cse23/hcmai/dataset/CLIP_features/L01/V001.npy
Loaded vector from /home/students/ttnhan-cse23/hcmai/dataset/CLIP_features/L01/V002.npy
Loaded vector from /home/students/ttnhan-cse23/hcmai/dataset/CLIP_features/L01/V003.npy
Loaded vector from /home/students/ttnhan-cse23/hcmai/dataset/CLIP_features/L01/V004.npy
Loaded vector from /home/students/ttnhan-cse23/hcmai/dataset/CLIP_features/L01/V005.npy


In [ ]:
vectors = np.vstack(vector_list_CLIP)
vector_dim = vectors.shape[1]
print(f"Loaded {vector.shape[0]} vectors of dimension {vector_dim}")

index = faiss.IndexFlatIP(vector_dim)

index.add(vectors)
print(index.ntotal)

Loaded 933 vectors of dimension 768
4275


In [28]:
# Save index
faiss_index_dir = "/home/students/ttnhan-cse23/hcmai/dict/faiss_normal_ViT.bin"
faiss.write_index(index, faiss_index_dir)

In [ ]:
image_folder = 

valid_extensions = (".jpg", ".png")

# Recursively get all image paths
image_paths = sorted(glob.glob(os.path.join(image_folder, "**", "**", "*.jpg"), recursive=True))

# Load and preprocess images
images = [preprocess(Image.open(img_path)).unsqueeze(0).to(device) for img_path in image_paths]

text_query = "vào buổi sáng, rất nhiều người đứng trước các tòa nhà cao tầng"
display_similar_images(image_paths, index, text_query, num_images=5)   